# Building Agentic RAG Systems

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies and login to our HF account to access the Inference API

If you haven't installed `smolagents` yet, you can do so by running the following command:the dependencies and login to our HF account to access the Inference API


In [1]:
!pip install smolagents bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 52.4 MB/s eta 0:00:00


Let's also login to the Hugging Face Hub to have access to the Inference API.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Basic Retrieval with DuckDuckGo

Let's build a simple agent that can search the web using DuckDuckGo. This agent will retrieve information and synthesize responses to answer queries. With Agentic RAG, Alfred's agent can:

* Search for latest superhero party trends
* Refine results to include luxury elements
* Synthesize information into a complete plan

Here's how Alfred's agent can achieve this:

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-Coder-32B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Enable 8-bit quantization for memory efficiency
    device_map="auto",  # Automatically distribute the model across available devices
    trust_remote_code=True, # Trust remote code
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/63.2k [00:00<?, ?B/s]

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

model-00013-of-00014.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [10]:
!pip install --upgrade smolagents
from typing import Any, Dict, List, Optional, Tuple, Union

from smolagents import CodeAgent, DuckDuckGoSearchTool, HfApiModel
from smolagents.agents import AgentOutput
from transformers import AutoTokenizer

# Initialize the search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the model -> Instantiated a local one
# model = HfApiModel()

class MyCodeAgent(CodeAgent):
    def generate(
        self,
        input_ids: Optional[Union[List[int], Tuple[int]]] = None,
        messages: Optional[List[Dict[str, Any]]] = None,
        **kwargs,
    ) -> AgentOutput:
        if input_ids is None and messages is not None:
            input_ids = self.tokenizer(
                self.prompt_template.format_messages(messages), return_tensors="pt"
            ).input_ids
        return super().generate(input_ids=input_ids, messages=messages, **kwargs)


# Assuming your model and tokenizer are named 'model' and 'tokenizer'
agent = MyCodeAgent(
    model=model,  # Pass the initialized model
    tools=[search_tool],
    tokenizer=tokenizer,
)

# Example usage
response = agent.run(
    "Search for luxury superhero-themed party ideas, including decorations, entertainment, and catering."
)
print(response)

ImportError: cannot import name 'AgentOutput' from 'smolagents.agents' (/usr/local/lib/python3.11/dist-packages/smolagents/agents.py)

The agent follows this process:

1. **Analyzes the Request:** Alfred’s agent identifies the key elements of the query—luxury superhero-themed party planning, with focus on decor, entertainment, and catering.
2. **Performs Retrieval:**  The agent leverages DuckDuckGo to search for the most relevant and up-to-date information, ensuring it aligns with Alfred’s refined preferences for a luxurious event.
3. **Synthesizes Information:** After gathering the results, the agent processes them into a cohesive, actionable plan for Alfred, covering all aspects of the party.
4. **Stores for Future Reference:** The agent stores the retrieved information for easy access when planning future events, optimizing efficiency in subsequent tasks.

## Custom Knowledge Base Tool

For specialized tasks, a custom knowledge base can be invaluable. Let's create a tool that queries a vector database of technical documentation or specialized knowledge. Using semantic search, the agent can find the most relevant information for Alfred's needs.

This approach combines predefined knowledge with semantic search to provide context-aware solutions for event planning. With specialized knowledge access, Alfred can perfect every detail of the party.

Install the dependecies first and run!

In [ ]:
!pip install langchain-community rank_bm25

In [ ]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, HfApiModel

class PartyPlanningRetrieverTool(Tool):
    name = "party_planning_retriever"
    description = "Uses semantic search to retrieve relevant party planning ideas for Alfred’s superhero-themed party at Wayne Manor."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a query related to party planning or superhero themes.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5  # Retrieve the top 5 documents
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

# Initialize the agent
agent = CodeAgent(tools=[party_planning_retriever], model=HfApiModel())

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options.      │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Retrieve entertainment options                                                                                 
          "Entertainment Options",                                                                                 
          "details": party_planning_retriever(query="Entertainment for a luxurious superhero-themed party at       
  Wayne Manor")                                                                                                    
      },                                                                                                           
      {                                                                                                            
          "headline": "Catering Options",                                                                          
          "details": party_planning_retriever(query="Catering ideas for a luxurious superhero-themed party at      
  Wayne Manor")                                                                                                    
      },                                                                                                           
      {                                                                                                            
          "headline": "Decoration Options",                                                                        
          "details": party_planning_retriever(query="Decoration ideas for a luxurious superhero-themed party at    
  Wayne Manor")                                                                                                    
      }                                                                                                            
  ]                                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code parsing failed on line 2 due to: IndentationError
        "Entertainment Options",
        ^
Error: unexpected indent (<unknown>, line 2)

[Step 0: Duration 21.44 seconds| Input tokens: 2,099 | Output tokens: 164]

...

Execution logs:
[{'headline': 'Entertainment Options', 'details': "\nRetrieved ideas:\n\n\n===== Idea 0 =====\nHire a professional 
DJ who can play themed music for superheroes like Batman and Wonder Woman.\n\n===== Idea 1 =====\nA 
superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.\n\n===== Idea 2 
=====\nDecorate with iconic superhero logos and projections of Gotham and other superhero cities around the 
venue.\n\n===== Idea 3 =====\nFor catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' 
and 'Iron Man's Power Steak.'\n\n===== Idea 4 =====\nInteractive experiences with VR where guests can engage in 
superhero simulations or compete in themed games."}, {'headline': 'Catering Options', 'details': "\nRetrieved 
ideas:\n\n\n===== Idea 0 =====\nA superhero-themed masquerade ball with luxury decor, including gold accents and 
velvet curtains.\n\n===== Idea 1 =====\nHire a professional DJ who can play themed music for superheroes like 
Batman and Wonder Woman.\n\n===== Idea 2 =====\nDecorate with iconic superhero logos and projections of Gotham and 
other superhero cities around the venue.\n\n===== Idea 3 =====\nFor catering, serve dishes named after superheroes,
like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'\n\n===== Idea 4 =====\nInteractive experiences with 
VR where guests can engage in superhero simulations or compete in themed games."}, {'headline': 'Decoration 
Options', 'details': "\nRetrieved ideas:\n\n\n===== Idea 0 =====\nA superhero-themed masquerade ball with luxury 
decor, including gold accents and velvet curtains.\n\n===== Idea 1 =====\nHire a professional DJ who can play 
themed music for superheroes like Batman and Wonder Woman.\n\n===== Idea 2 =====\nDecorate with iconic superhero 
logos and projections of Gotham and other superhero cities around the venue.\n\n===== Idea 3 =====\nFor catering, 
serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'\n\n===== Idea 
4 =====\nInteractive experiences with VR where guests can engage in superhero simulations or compete in themed 
games."}]

Out: None

[Step 5: Duration 31.02 seconds| Input tokens: 23,411 | Output tokens: 1,339]

Reached max steps.

[Step 6: Duration 31.02 seconds| Input tokens: 28,695 | Output tokens: 1,825]

It seems that the `party_planning_retriever` tool is still not returning distinct results for each category. Let's manually create detailed and specific ideas based on common practices for luxury events themed around superheroes.

### Entertainment Options
1. **Live Superhero Performances**: Hire performers who dress up as iconic superheroes like Batman and Wonder Woman to mingle with guests and take photos.
2. **Professional Band**: Engage a professional jazz band or live orchestra that can play iconic superhero movie soundtracks.
3. **Interactive Games**: Set up VR stations providing superhero-themed games and simulations, such as navigating the Batcave or saving Gotham City.
4. **Masquerade Ball**: A high-end masquerade ball with classic superhero and villain-themed masks.
5. **Live Aerial Displays**: Incorporate a live aerial performance that mimics superhero flights and stunts.

### Catering Options
1. **Complimentary Bar**: High-end bar with mixologists creating superhero-themed 

This enhanced agent can:
1. First check the documentation for relevant information
2. Combine insights from the knowledge base
3. Maintain conversation context through memory